# PyBN examples

In [1]:
import numpy as np
import pandas as pd
import lark

import pybn
import pybn.examples
import pybn.oobn

from IPython.display import display, HTML

print(f'Using PyBN version: {pybn.__version__}')
print(f'Using Pandas version: {pd.__version__}')

%load_ext autoreload
%autoreload 2

Using PyBN version: 2019-03-18 21:07
Using Pandas version: 0.24.1


## Playing with Factors

In [2]:
# Get the Factors for the Sprinkler network
fA, fB_A, fC_A, fD_BC, fE_C = pybn.examples.get_sprinkler_factors()

In [3]:
# Multiplying the factor with a *prior* with a *conditional* distribution, yields
# a *joint* distribution.
fAB = fA * fB_A
display(fAB)

factor(A,B)
A   B 
a1  b1    0.12
    b0    0.48
a0  b1    0.30
    b0    0.10
dtype: float64

In [4]:
# By summing out A, we'll get the prior over B
fAB.sum_out('A')

factor(B)
B
b0    0.58
b1    0.42
dtype: float64

In [5]:
# To get to this prior directly, we could also have taken the dot (matrix) product,
# but we'd have to turn B_A into a matrix.
fA.dot(fB_A.unstack())

B
b0    0.58
b1    0.42
dtype: float64

In [6]:
# By putting the factors in a Bag, we can reason over them
bag = pybn.Bag(factors=[fA, fB_A, fC_A, fD_BC, fE_C])

In [7]:
bag.scope

{'A', 'B', 'C', 'D', 'E'}

In [8]:
# Compute the factor over C
bag.eliminate(['C'])

factor(C)
C
c0    0.48
c1    0.52
dtype: float64

In [9]:
# Compute the (unnormalized) factor over C and A=a1
bag.eliminate(['C'], {'A': 'a1'})

factor(C)
C
c0    0.12
c1    0.48
dtype: float64

In [10]:
# From this we can deduce that P(A=a1) = 0.6
Pa1 = bag.eliminate(['C'], {'A': 'a1'}).sum()
f'P(A=a1) = {Pa1:.2}'

'P(A=a1) = 0.6'

In [11]:
# Normalizing is easy and gives the conditional probability P(C|A=a1)
bag.eliminate(['C'], {'A': 'a1'}).normalize()

factor(C)
C
c0    0.2
c1    0.8
dtype: float64

In [12]:
# We can also compute the joint distribution over A and C. 
bag.eliminate(['A', 'C'])

factor(A,C)
A   C 
a0  c0    0.36
    c1    0.04
a1  c0    0.12
    c1    0.48
dtype: float64

In [13]:
# Calling `unstack` makes things a little easier on the eyes.
bag.eliminate(['A', 'C']).unstack()

C,c0,c1
A,,
a0,0.36,0.04
a1,0.12,0.48


In [14]:
# Or even over the entire joint distribution (bag.scope holds/returns a list of all 
# variables in the bag). Don't do this for anything other than really simple networks
# as the number of entries in the JPT will blow up quickly.
bag.eliminate(bag.scope).reorder_scope().sort_index()

factor(A,B,C,D,E)
A   B   C   D   E 
a0  b0  c0  d0  e0    0.09000
                e1    0.00000
            d1  e0    0.00000
                e1    0.00000
        c1  d0  e0    0.00060
                e1    0.00140
            d1  e0    0.00240
                e1    0.00560
    b1  c0  d0  e0    0.02700
                e1    0.00000
            d1  e0    0.24300
                e1    0.00000
        c1  d0  e0    0.00045
                e1    0.00105
            d1  e0    0.00855
                e1    0.01995
a1  b0  c0  d0  e0    0.09600
                e1    0.00000
            d1  e0    0.00000
                e1    0.00000
        c1  d0  e0    0.02304
                e1    0.05376
            d1  e0    0.09216
                e1    0.21504
    b1  c0  d0  e0    0.00240
                e1    0.00000
            d1  e0    0.02160
                e1    0.00000
        c1  d0  e0    0.00144
                e1    0.00336
            d1  e0    0.02736
                e1    0.06384
dty

## Querying the Student network

In [15]:
# Load the network from the examples
bn = pybn.examples.get_student_network()

In [16]:
bn['I']

I,i0,i1
,0.7,0.3


In [17]:
# Retrieving a Node/CPT can be done through indexes
# Prior probability for node I (intelligence):
display(bn['I'])

# Conditional probability for node S (SAT):
display(bn['S'])

I,i0,i1
,0.7,0.3


S,s0,s1
I,,
i0,0.95,0.05
i1,0.20,0.80


In [18]:
# Compute the prior probality for a student to get a good result on the SAT
bn.P('S')

S,s0,s1
,0.725,0.275


In [19]:
# Compute the conditional probability that a student gets a good grade, 
# given (s)he's intelligent
bn.P('G|I=i1')

G,g1,g2,g3
I,,,
i1,0.74,0.168,0.092


In [20]:
bn.P('G|I')

G,g1,g2,g3
I,,,
i0,0.20,0.340,0.460
i1,0.74,0.168,0.092
